In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
import pandas as pd
import elevation
import elevation.prediction_pipeline as pp
import elevation.aggregation
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
np.random.seed(123) 
import scipy.stats as st
import pickle
#import seaborn
import matplotlib
import azimuth.util
import sklearn
import copy
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.linear_model
from sklearn.grid_search import GridSearchCV
import elevation.aggregation
reload(elevation.aggregation)

<module 'elevation.aggregation' from 'c:\dev\elevation\elevation\aggregation.pyc'>

In [87]:
original_new_gecko_df = pd.read_csv('C:\\dev\\CRISPR\\data\\Mudra_clean.2017-04-10\\GeCKOv2_essential_nonessential_v4.csv')

In [88]:
df = original_new_gecko_df
np.sum((df["Non-Essential genes"] == "Y") & (df["Essential genes"] == "Y"))

363

In [89]:
new_gecko_df = original_new_gecko_df
print new_gecko_df["Assay Type"].unique().tolist()
print new_gecko_df["Non-Essential genes"].unique().tolist()

data = new_gecko_df
isviability = data["Assay Type"].apply(lambda x: "viability" in x if isinstance(x, basestring) else False)
isnot_essential = data['Non-Essential genes'] == "Y"
viable_and_essential = (isviability & isnot_essential) | (~isviability)
new_gecko_df = data[viable_and_essential]

celltypes = new_gecko_df['Cell Type'].unique()
original_celltypes = celltypes
celltypes = map(lambda x: x.split("_")[0], celltypes)
celltype_map = dict(zip(celltypes, original_celltypes))

['Cell viability (final - initial)']
['N', 'Y']


In [69]:
new_gecko_df.shape

(160578, 29)

In [93]:
a = new_gecko_df[new_gecko_df['Cell Type'] == celltype_map['A375']]
print a['Cell Type'].unique()
a = a.dropna(how="any", subset=['sgRNA context sequence'])
b = pd.read_csv('../../CRISPR/data/offtarget/3MM_michael/GeCKOv2_Non_essentials_Achilles_A375.txt', delimiter='\t')

['A375_SKIN']


In [95]:
a['full_seq'] = a['sgRNA context sequence'].apply(lambda x: x[4:20+4+3])
b['full_seq'] = b['sgRNA context sequence'].apply(lambda x: x[4:20+4+3])
ab = pd.merge(a, b, how='outer', left_on='full_seq', right_on='full_seq', indicator=True)

In [101]:
print a.shape[0], b.shape[0], ab.shape[0]

4763 4697 4773


In [104]:
num_nan = 0
for seq, x, y in ab[ab['_merge'] == 'both'][['full_seq', 'sgRNA measured value_x', 'sgRNA measured value_y']].values:
    if np.isnan(x) or np.isnan(y):
        num_nan += 1
        continue
    assert np.allclose(x, -y), "%f %f" % (x, y)
print num_nan

0


In [105]:
print(len(a), len(b))

(4763, 4697)


In [106]:
print ab['_merge'].unique()
print ab[ab['full_seq'] == 'AAGTCAACACCAGATACAAGAGG'].shape
print ab[ab['full_seq'] == 'AAGTCAACACCAGATACAAGAGG'].iloc[0][['sgRNA measured value_x', 'sgRNA measured value_y']]
print ab[ab['full_seq'] == 'AAGTCAACACCAGATACAAGAGG'].iloc[0][['sgRNA context sequence_x', 'sgRNA context sequence_y']]

[both, left_only, right_only]
Categories (3, object): [both, left_only, right_only]
(1, 56)
sgRNA measured value_x    1.59897
sgRNA measured value_y        NaN
Name: 975, dtype: object
sgRNA context sequence_x    ACTTAAGTCAACACCAGATACAAGAGGAACAGGAGG
sgRNA context sequence_y                                     NaN
Name: 975, dtype: object


In [6]:
ab[ab['full_seq'] == 'ACTGCTGATCAGCTAACTACTGG'].iloc[0]

Species_x                                                                     human
Build_x                                                                      GRCh38
Chromosome Number_x                                                              13
Gene Strand_x                                                                     +
Ensembl Gene ID_x                                                   ENSG00000132972
Gene Symbol_x                                                               "RNF17"
Ensembl Transcript ID_x                                             ENST00000255324
Transcript TSS_x                                                        2.47642e+07
sgRNA Start_x                                                           2.47783e+07
sgRNA Stop_x                                                            2.47784e+07
sgRNA Sequence_x                                               ACTGCTGATCAGCTAACTAC
sgRNA context sequence_x                       GAAGACTGCTGATCAGCTAACTACTGGTT